# AMBRS x CAMP multi-mechanism demo (H2SO4 example + PPE checks)

This notebook drives CAMP chemistry from AMBRS in a way that is:
- **General** (use a template mechanism, a file list, a directory of JSONs, explicit files, or inline components)
- **Minimal** (all CAMP files are written under each scenario directory)
- **Testable** (side-by-side comparisons and conservation checks across a PPE)

The default example uses a simple **H2SO4** unimolecular sink (first-order rate) to mimic condensation. You can swap mechanisms via one line at the top.


In [1]:

# --- User settings (edit as needed) ---
MAM4_EXE = "mam4"           # or absolute path
PARTMC_EXE = "partmc"       # or absolute path

ROOT = "runs_demo_camp"     # root folder for all runs
BASELINE_TAG = "baseline"
CAMP_TAG     = "camp"

# H2SO4 sink timescale (for the template): k = 1/tau
H2SO4_TAU_SECONDS = 3600.0        # 1 hour
H2SO4_K = 1.0 / H2SO4_TAU_SECONDS

# location of libcamp (optional; used if libcamp is not discoverable)
LIBCAMP_DIR = None  # e.g., "/Users/you/miniforge3/envs/ambrs_camp_sync/lib"

# Mechanism source for CAMP (choose ONE; default is the template)
MECH_SOURCE = {
    # Option A (default): built-in H2SO4 sink template
    "template": {"name": "h2so4_sink", "rate_s_inv": H2SO4_K}

    # Option B: Use a pre-existing CAMP file-list JSON
    # "file_list": "/abs/path/to/camp_file_list.json",

    # Option C: Use all JSON files from a mechanism directory (scanned, sorted)
    # "mechanism_dir": "/abs/path/to/camp_mech_dir",

    # Option D: Provide an explicit ordered list of JSON files
    # "mechanism_files": ["/abs/path/species.json", "/abs/path/aero_phase.json", "/abs/path/mechanism.json"],

    # Option E: Provide inline components (will be written locally under each scenario)
    # "components": {
    #     "species": [{"type":"CHEM_SPEC","name":"X","phase":"GAS"}],
    #     "aero_phases": [{"type":"AERO_PHASE","name":"phase0","species":["X"]}],
    #     "mechanisms": [{"type":"MECHANISM","name":"mech","reactions":[]}]
    # }
}


In [2]:

# Imports
import os, math, json
from pathlib import Path
import numpy as np
from netCDF4 import Dataset

import ambrs
from ambrs import mam4, partmc
from ambrs.aerosol import AerosolProcesses
from ambrs.ppe import Ensemble
from ambrs.runners import PoolRunner
from ambrs.camp import CampConfig  # generalized CAMP helper


## Create a CAMP driver (general helper, mechanism selected above)

In [5]:
# camp = CampConfig(lib_dir=LIBCAMP_DIR, source=MECH_SOURCE)
camp = CampConfig(
    mechanism_name="ambrs_h2so4_soag",
    # explicit_lib_dirs=(Path("/absolute/path/to/camp/lib"), Path(os.environ["CONDA_PREFIX"])/"lib")
)


## Build a small PPE (replace with your own PPE definition if desired)

In [ ]:

ensemble = ambrs.lhs(specification=spec, n=n)
processes_default = AerosolProcesses(
    gas_phase_chemistry=True, condensation=True, coagulation=True,
    nucleation=False, optics=False
)

# Replace with your PPE builder if desired (e.g., Ensemble.from_config(...))
# ensemble = Ensemble.default(n=3, seed=42)

dt = 300.0   # 5 minutes
nstep = 12   # 1 hour
width = max(1, int(math.log10(len(ensemble.scenarios))) + 1)


NameError: name 'spec' is not defined

## Run MAM4 — baseline (native condensation) vs CAMP (H2SO4 sink)

In [0]:

mam4_root = Path(ROOT)/"mam4"
(mam4_root/BASELINE_TAG).mkdir(parents=True, exist_ok=True)
(mam4_root/CAMP_TAG).mkdir(parents=True, exist_ok=True)

# Baseline MAM4 (native condensation ON, no CAMP)
mam4_native = mam4.AerosolModel(
    processes=AerosolProcesses(True, True, True, False, False),
    camp=None
)
inputs_native = [mam4_native.create_input(s, dt, nstep) for s in ensemble.scenarios]
PoolRunner(mam4_native, MAM4_EXE, str(mam4_root/BASELINE_TAG), num_processes=1).run(inputs_native)

# CAMP MAM4 (native condensation OFF; CAMP enabled)
mam4_with_camp = mam4.AerosolModel(
    processes=AerosolProcesses(True, False, True, False, False),
    camp=camp
)
inputs_camp = [mam4_with_camp.create_input(s, dt, nstep) for s in ensemble.scenarios]

# ensure per-scenario camp/ exists before run (nice-to-have)
for i, s in enumerate(ensemble.scenarios):
    scen_dir = mam4_root/CAMP_TAG/f"{i+1:0{width}d}"
    Path(scen_dir).mkdir(parents=True, exist_ok=True)
    camp.write_common_files(str(scen_dir), scenario=s)

PoolRunner(mam4_with_camp, MAM4_EXE, str(mam4_root/CAMP_TAG), num_processes=1).run(inputs_camp)


## Run PartMC — baseline (MOSAIC) vs CAMP (H2SO4 sink)

In [0]:

pm_root = Path(ROOT)/"partmc"
(pm_root/BASELINE_TAG).mkdir(parents=True, exist_ok=True)
(pm_root/CAMP_TAG).mkdir(parents=True, exist_ok=True)

# Baseline PartMC (MOSAIC ON, no CAMP)
pm_native = partmc.AerosolModel(
    processes=AerosolProcesses(False, True, True, False, False),
    run_type='particle', n_part=20000, n_repeat=1, camp=None
)
pm_inputs_native = [pm_native.create_input(s, dt, nstep) for s in ensemble.scenarios]
PoolRunner(pm_native, PARTMC_EXE, str(pm_root/BASELINE_TAG), num_processes=1).run(pm_inputs_native)

# CAMP PartMC (MOSAIC OFF; CAMP enabled)
pm_camp = partmc.AerosolModel(
    processes=AerosolProcesses(False, False, True, False, False),
    run_type='particle', n_part=20000, n_repeat=1, camp=camp
)
pm_inputs_camp = [pm_camp.create_input(s, dt, nstep) for s in ensemble.scenarios]

# ensure per-scenario camp/ exists before run
for i, s in enumerate(ensemble.scenarios):
    scen_dir = pm_root/CAMP_TAG/f"{i+1:0{width}d}"
    Path(scen_dir).mkdir(parents=True, exist_ok=True)
    camp.write_common_files(str(scen_dir), scenario=s)

PoolRunner(pm_camp, PARTMC_EXE, str(pm_root/CAMP_TAG), num_processes=1).run(pm_inputs_camp)


## Readers and diagnostics (H2SO4, SO4, and total sulfur proxy)

In [0]:

def read_mam4_timeseries(run_root: Path, scenario_name: str):
    nc = Dataset(str(run_root/scenario_name/'mam_output.nc'))
    out = {}
    out["time"] = np.array(nc.variables["time"][:]) if "time" in nc.variables else None
    if "h2so4_gas" in nc.variables:
        out["h2so4_gas"] = np.array(nc.variables["h2so4_gas"][:])
    else:
        cand = [v for v in nc.variables if "h2so4" in v.lower() and "gas" in v.lower()]
        out["h2so4_gas"] = np.array(nc.variables[cand[0]][:]) if cand else None
    so4 = None
    for v in nc.variables:
        vl = v.lower()
        if "so4" in vl and ("aer" in vl or "mode" in vl or "pm" in vl) and "gas" not in vl:
            arr = np.array(nc.variables[v][:])
            so4 = arr if so4 is None else so4 + arr
    out["so4_aer"] = so4
    return out

def read_partmc_timeseries(run_root: Path, scenario_name: str):
    outdir = run_root/scenario_name/"out"
    files = sorted([f for f in os.listdir(outdir) if f.endswith(".nc")])
    if not files: return {"time":None,"h2so4_gas":None,"so4_aer":None}
    times, h2so4_series = [], []
    so4_series = None
    for f in files:
        nc = Dataset(str(outdir/f))
        try:
            tstep = int(f.split("_")[-1].split(".")[0])
        except Exception:
            tstep = len(times)
        times.append(tstep)
        if "gas_species" in nc.variables and "gas_mixing_ratio" in nc.variables:
            names = nc.variables["gas_species"].names.split(",")
            data = np.array(nc.variables["gas_mixing_ratio"][:])
            if "H2SO4" in names:
                idx = names.index("H2SO4")
                h2so4_series.append(data[idx])
            else:
                h2so4_series.append(np.nan)
        if so4_series is None:
            so4_series = []
        found_so4 = False
        for v in nc.variables:
            vl = v.lower()
            if "so4" in vl and ("aer" in vl or "mass" in vl or "pm" in vl) and "gas" not in vl:
                arr = np.array(nc.variables[v][:])
                val = float(np.atleast_1d(arr)[-1]) if arr.size else np.nan
                so4_series.append(val); found_so4=True; break
        if not found_so4: so4_series.append(np.nan)
    return {"time": np.array(times),
            "h2so4_gas": np.array(h2so4_series) if h2so4_series else None,
            "so4_aer": np.array(so4_series) if so4_series is not None else None}

def rel_err(a, b):
    a = np.asarray(a); b = np.asarray(b)
    return np.abs(b - a) / np.maximum(1e-30, np.abs(a))

def summarize_pair(model_label, root_base: Path, root_camp: Path, scenario_name: str):
    r = {"model": model_label, "scenario": scenario_name}
    reader = read_mam4_timeseries if model_label.lower()=="mam4" else read_partmc_timeseries
    base = reader(root_base, scenario_name)
    camp = reader(root_camp, scenario_name)
    # H2SO4 similarity
    if base["h2so4_gas"] is not None and camp["h2so4_gas"] is not None:
        n = min(len(base["h2so4_gas"]), len(camp["h2so4_gas"]))
        r["h2so4_rel_err_mean"] = float(np.nanmean(rel_err(base["h2so4_gas"][:n], camp["h2so4_gas"][:n])))
        r["h2so4_base_final"] = float(base["h2so4_gas"][:n][-1]); r["h2so4_camp_final"] = float(camp["h2so4_gas"][:n][-1])
    else:
        r["h2so4_rel_err_mean"] = float("nan")
    # SO4 and total sulfur proxy (within-model)
    if base["so4_aer"] is not None and camp["so4_aer"] is not None and base["h2so4_gas"] is not None and camp["h2so4_gas"] is not None:
        n2 = min(len(base["so4_aer"]), len(camp["so4_aer"]))
        r["so4_rel_err_mean"] = float(np.nanmean(rel_err(base["so4_aer"][:n2], camp["so4_aer"][:n2])))
        nb = min(n, n2)
        tb = np.asarray(base["h2so4_gas"][:nb]) + np.asarray(base["so4_aer"][:nb])
        tc = np.asarray(camp["h2so4_gas"][:nb]) + np.asarray(camp["so4_aer"][:nb])
        r["totalS_base_span"] = float(np.nanmax(tb) - np.nanmin(tb))
        r["totalS_camp_span"] = float(np.nanmax(tc) - np.nanmin(tc))
    else:
        r["so4_rel_err_mean"] = float("nan")
    return r


## Evaluate checks across the ensemble

In [0]:

results = []
# MAM4 comparison
for i in range(len(ensemble.scenarios)):
    name = f"{i+1:0{width}d}"
    results.append(summarize_pair("MAM4",
                                  Path(ROOT)/"mam4"/BASELINE_TAG,
                                  Path(ROOT)/"mam4"/CAMP_TAG,
                                  name))
# PartMC comparison
for i in range(len(ensemble.scenarios)):
    name = f"{i+1:0{width}d}"
    results.append(summarize_pair("PartMC",
                                  Path(ROOT)/"partmc"/BASELINE_TAG,
                                  Path(ROOT)/"partmc"/CAMP_TAG,
                                  name))

results
